In [3]:
import duckdb
import pandas as pd

# 1. 准备一些示例数据（实际项目里可以来自 CSV/接口等）
prices = pd.DataFrame(
    {
        "symbol": ["AAPL", "AAPL", "MSFT", "MSFT"],
        "date": pd.to_datetime(
            ["2024-01-02", "2024-01-03", "2024-01-02", "2024-01-03"]
        ),
        "close": [185.1, 187.3, 330.2, 333.8],
        "volume": [100_000, 120_000, 90_000, 110_000],
    }
)

# 2. 连接到 DuckDB 数据库文件（不存在会自动创建）
con = duckdb.connect(r"data/data.duckdb")

# 3. 将 pandas DataFrame 写入数据库（存）
con.register("prices_df", prices)  # 把 DataFrame 注册成一个临时视图
con.execute("CREATE OR REPLACE TABLE prices AS SELECT * FROM prices_df")

# 4. 追加新数据（可选）
con.begin()
new_rows = pd.DataFrame(
    {
        "symbol": ["AAPL"],
        "date": pd.to_datetime(["2024-01-04"]),
        "close": [189.0],
        "volume": [130_000],
    }
)
con.register("new_rows_df", new_rows)
con.execute("INSERT INTO prices SELECT * FROM new_rows_df")
con.commit()

# 5. 查询（取）
result_df = con.execute(
    """
    SELECT symbol, date, close
    FROM prices
    WHERE symbol = 'AAPL'
    ORDER BY date
"""
).df()
print(result_df)

# 6. 导出为 Parquet（可选：便于与其他工具共享）
con.execute("COPY (SELECT * FROM prices) TO 'data\prices.parquet' (FORMAT PARQUET)")

# 7. 从 Parquet 直接读，不用先导入表（DuckDB 支持直接 SELECT 文件）
parquet_df = con.execute(
    "SELECT * FROM 'data\prices.parquet' WHERE volume > 100000"
).df()
print(parquet_df)

# 8.定期清理
con.execute("VACUUM")
con.execute("ANALYZE")
# 9. 关闭连接
con.close()

  symbol       date  close
0   AAPL 2024-01-02  185.1
1   AAPL 2024-01-03  187.3
2   AAPL 2024-01-04  189.0
  symbol       date  close  volume
0   AAPL 2024-01-03  187.3  120000
1   MSFT 2024-01-03  333.8  110000
2   AAPL 2024-01-04  189.0  130000
